# YOLOv8 의류 객체 탐지 모델 학습
## Multimodal Coordinator - Fashionpedia 기반

이 노트북은 다음을 수행합니다:
1. 환경 설정 및 패키지 설치
2. Fashionpedia 데이터셋 다운로드 (승인 불필요!)
3. YOLO 형식으로 변환 (6:2:2 = train/val/test)
4. YOLOv8 학습
5. 평가 및 모델 다운로드

---
**Fashionpedia**: ~48,000장, CC BY 4.0 라이선스, 승인 불필요

**런타임 설정**: `런타임 > 런타임 유형 변경 > GPU (T4)` 선택

## Step 1. 환경 설정

In [ ]:
# GPU 확인
!nvidia-smi

In [ ]:
# 필요 패키지 설치
!pip install -q ultralytics pillow

In [ ]:
import os
import json
import shutil
import random
from pathlib import Path
from collections import defaultdict
from PIL import Image
from google.colab import drive

# Google Drive 마운트 (결과 저장용)
drive.mount('/content/drive')

print("환경 설정 완료!")

## Step 2. Fashionpedia 데이터셋 다운로드

AWS S3에서 직접 다운로드합니다. (승인 불필요)
- 이미지: train (~7GB) + val (~1GB)
- 어노테이션: JSON 2개 (~200MB)

In [ ]:
# ── 작업 디렉토리 생성 ──
!mkdir -p /content/fashionpedia/annotations

# ── 어노테이션 다운로드 (빠름, ~1분) ──
print("어노테이션 다운로드 중...")
!wget -q -O /content/fashionpedia/annotations/instances_attributes_train2020.json \
  https://s3.amazonaws.com/ifashionist-dataset/annotations/instances_attributes_train2020.json
!wget -q -O /content/fashionpedia/annotations/instances_attributes_val2020.json \
  https://s3.amazonaws.com/ifashionist-dataset/annotations/instances_attributes_val2020.json

print("어노테이션 다운로드 완료!")
!ls -lh /content/fashionpedia/annotations/

In [ ]:
# ── 이미지 다운로드 (약 10~20분 소요) ──
print("Train 이미지 다운로드 중... (약 7GB, 10~15분)")
!wget -q -O /content/train2020.zip \
  https://s3.amazonaws.com/ifashionist-dataset/images/train2020.zip

print("Val 이미지 다운로드 중... (약 1GB, 2~3분)")
!wget -q -O /content/val_test2020.zip \
  https://s3.amazonaws.com/ifashionist-dataset/images/val_test2020.zip

print("\n다운로드 완료! 압축 해제 중...")
!unzip -q /content/train2020.zip -d /content/fashionpedia/
!unzip -q /content/val_test2020.zip -d /content/fashionpedia/

# 다운로드 파일 삭제 (디스크 절약)
!rm /content/train2020.zip /content/val_test2020.zip

print("압축 해제 완료!")

In [ ]:
# ── 데이터셋 구조 확인 ──
print("=== 폴더 구조 ===")
!ls /content/fashionpedia/

print("\n=== train 이미지 수 ===")
!ls /content/fashionpedia/train/ | wc -l

print("\n=== test 이미지 수 ===")
!ls /content/fashionpedia/test/ | wc -l

## Step 3. YOLO 형식으로 변환 (6:2:2 분할)

In [ ]:
# ── Fashionpedia → YOLO 변환 코드 ──

# Fashionpedia 카테고리(0~26) → YOLO 클래스(0~4) 매핑
# 27번 이상은 의류 부분(소매, 주머니 등)이므로 제외
CATEGORY_MAP = {
    0: 0,    # shirt, blouse          → top
    1: 0,    # top, t-shirt, sweatshirt → top
    2: 0,    # sweater                → top
    3: 2,    # cardigan               → outer
    4: 2,    # jacket                 → outer
    5: 0,    # vest                   → top
    6: 1,    # pants                  → bottom
    7: 1,    # shorts                 → bottom
    8: 1,    # skirt                  → bottom
    9: 2,    # coat                   → outer
    10: 3,   # dress                  → dress
    11: 3,   # jumpsuit               → dress
    12: 2,   # cape                   → outer
    13: 4,   # glasses                → acc
    14: 4,   # hat                    → acc
    15: 4,   # headband, hair acc     → acc
    16: 4,   # tie                    → acc
    17: 4,   # glove                  → acc
    18: 4,   # watch                  → acc
    19: 4,   # belt                   → acc
    20: 4,   # leg warmer             → acc
    21: 1,   # tights, stockings      → bottom
    22: 4,   # sock                   → acc
    23: 4,   # shoe                   → acc
    24: 4,   # bag, wallet            → acc
    25: 4,   # scarf                  → acc
    26: 4,   # umbrella               → acc
}

YOLO_CLASSES = {0: "top", 1: "bottom", 2: "outer", 3: "dress", 4: "acc"}


def convert_coco_bbox_to_yolo(bbox, img_width, img_height):
    """COCO [x_min, y_min, w, h] → YOLO [x_center, y_center, w, h] 정규화"""
    x_min, y_min, w, h = bbox
    x_center = max(0.0, min(1.0, (x_min + w / 2.0) / img_width))
    y_center = max(0.0, min(1.0, (y_min + h / 2.0) / img_height))
    w_norm = max(0.0, min(1.0, w / img_width))
    h_norm = max(0.0, min(1.0, h / img_height))
    return x_center, y_center, w_norm, h_norm


def load_annotations(json_path):
    """COCO JSON 로드 → 이미지별 어노테이션 그룹화"""
    print(f"  로딩: {json_path}")
    with open(json_path, "r") as f:
        data = json.load(f)

    images = {img["id"]: img for img in data["images"]}

    anns_by_image = defaultdict(list)
    for ann in data["annotations"]:
        cat_id = ann["category_id"]
        if cat_id in CATEGORY_MAP:
            anns_by_image[ann["image_id"]].append(ann)

    print(f"  이미지: {len(images)}장 | 유효 어노테이션: {sum(len(v) for v in anns_by_image.values())}개")
    return images, anns_by_image


def convert_and_save(images, anns, image_ids, img_dirs, dst_dir, split_name):
    """이미지 ID 목록을 YOLO 형식으로 변환"""
    img_dst = Path(dst_dir) / split_name / "images"
    label_dst = Path(dst_dir) / split_name / "labels"
    img_dst.mkdir(parents=True, exist_ok=True)
    label_dst.mkdir(parents=True, exist_ok=True)

    converted = 0
    skipped = 0

    for i, img_id in enumerate(image_ids):
        img_info = images[img_id]
        file_name = img_info["file_name"]
        img_w = img_info["width"]
        img_h = img_info["height"]

        # 이미지 파일 찾기 (여러 폴더 탐색)
        img_path = None
        for d in img_dirs:
            p = Path(d) / file_name
            if p.exists():
                img_path = p
                break

        if img_path is None or img_id not in anns:
            skipped += 1
            continue

        # YOLO 라벨 생성
        lines = []
        for ann in anns[img_id]:
            yolo_class = CATEGORY_MAP[ann["category_id"]]
            bbox = ann["bbox"]
            if bbox[2] <= 0 or bbox[3] <= 0:
                continue
            x_c, y_c, w, h = convert_coco_bbox_to_yolo(bbox, img_w, img_h)
            lines.append(f"{yolo_class} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}")

        if not lines:
            skipped += 1
            continue

        # 심볼릭 링크 (디스크 절약)
        link_path = img_dst / file_name
        if not link_path.exists():
            os.symlink(img_path.resolve(), link_path)

        # 라벨 저장
        label_name = Path(file_name).stem + ".txt"
        with open(label_dst / label_name, "w") as f:
            f.write("\n".join(lines))

        converted += 1

        if (i + 1) % 5000 == 0:
            print(f"    진행: {i+1}/{len(image_ids)}")

    print(f"  [{split_name.upper()}] 변환: {converted}장 | 스킵: {skipped}장")
    return converted


print("변환 함수 정의 완료!")

In [ ]:
# ── 변환 실행 ──
SRC_DIR = "/content/fashionpedia"
DST_DIR = "/content/yolo_dataset"
SEED = 42
SPLIT_RATIO = (0.6, 0.2, 0.2)  # train / val / test

random.seed(SEED)

print("Fashionpedia → YOLO 변환 시작\n")

# 1. 어노테이션 로드
all_images = {}
all_anns = defaultdict(list)

for json_name in [
    "annotations/instances_attributes_train2020.json",
    "annotations/instances_attributes_val2020.json",
]:
    json_path = Path(SRC_DIR) / json_name
    if json_path.exists():
        imgs, anns = load_annotations(str(json_path))
        all_images.update(imgs)
        for img_id, ann_list in anns.items():
            all_anns[img_id].extend(ann_list)

# 2. 이미지 폴더 목록
img_dirs = [
    f"{SRC_DIR}/train",
    f"{SRC_DIR}/test",
]

# 3. 유효한 이미지 필터링
valid_ids = []
for img_id in all_images:
    if img_id not in all_anns:
        continue
    file_name = all_images[img_id]["file_name"]
    if any((Path(d) / file_name).exists() for d in img_dirs):
        valid_ids.append(img_id)

print(f"\n유효 이미지: {len(valid_ids)}장")

# 4. 6:2:2 분할
random.shuffle(valid_ids)
n = len(valid_ids)
n_train = int(n * SPLIT_RATIO[0])
n_val = int(n * SPLIT_RATIO[1])

train_ids = valid_ids[:n_train]
val_ids = valid_ids[n_train:n_train + n_val]
test_ids = valid_ids[n_train + n_val:]

print(f"분할: train={len(train_ids)} | val={len(val_ids)} | test={len(test_ids)}")

# 5. 변환
print()
for split_name, split_ids in [("train", train_ids), ("val", val_ids), ("test", test_ids)]:
    convert_and_save(all_images, all_anns, split_ids, img_dirs, DST_DIR, split_name)

print("\n변환 완료!")

In [ ]:
# ── data.yaml 생성 ──
data_yaml = f"""path: {DST_DIR}
train: train/images
val: val/images
test: test/images

names:
  0: top
  1: bottom
  2: outer
  3: dress
  4: acc

nc: 5
"""

yaml_path = os.path.join(DST_DIR, "data.yaml")
os.makedirs(DST_DIR, exist_ok=True)
with open(yaml_path, "w") as f:
    f.write(data_yaml)

print(f"data.yaml 생성: {yaml_path}")
print(data_yaml)

In [ ]:
# ── 클래스 분포 확인 ──
for split in ["train", "val", "test"]:
    label_dir = Path(DST_DIR) / split / "labels"
    if not label_dir.exists():
        continue

    counts = {i: 0 for i in range(5)}
    for label_file in label_dir.glob("*.txt"):
        with open(label_file, "r") as f:
            for line in f:
                parts = line.strip().split()
                if parts:
                    counts[int(parts[0])] += 1

    total = sum(counts.values())
    print(f"\n[{split.upper()}] (총 {total}개 객체)")
    for class_id, count in counts.items():
        pct = count / total * 100 if total > 0 else 0
        bar = "#" * int(pct / 2)
        print(f"  {YOLO_CLASSES[class_id]:>8s} (id={class_id}): {count:>6d} ({pct:5.1f}%) {bar}")

## Step 4. YOLOv8 학습

In [ ]:
from ultralytics import YOLO

# 모델 선택
# yolov8n: nano (빠름, 가벼움) ← 첫 실험에 추천
# yolov8s: small (균형)
# yolov8m: medium (높은 정확도)
model = YOLO("yolov8n.pt")

print(f"모델 로드 완료: {model.model_name}")

In [ ]:
# ── 학습 실행 ──
# Colab T4 기준: nano ~1~2시간, small ~2~3시간

results = model.train(
    data=f"{DST_DIR}/data.yaml",
    epochs=15,
    imgsz=640,
    batch=16,           
    device=0,
    project="/content/runs",
    name="clothing_v1",
    patience=5,        
    save=True,
    plots=True,
    workers=2,
)

## Step 5. 학습 결과 확인

In [ ]:
from IPython.display import Image as IPImage, display

result_dir = "/content/runs/clothing_v1"

# 학습 그래프
print("=== 학습 그래프 ===")
display(IPImage(filename=f"{result_dir}/results.png", width=800))

In [ ]:
# 혼동 행렬
print("=== 혼동 행렬 ===")
display(IPImage(filename=f"{result_dir}/confusion_matrix.png", width=600))

In [ ]:
# 검증 결과 시각화
print("=== 검증 결과 예시 ===")
display(IPImage(filename=f"{result_dir}/val_batch0_pred.jpg", width=800))

In [ ]:
# ── 최종 성능 평가 (Test 셋) ──
best_model = YOLO(f"{result_dir}/weights/best.pt")

# Validation 성능
print("=== Validation 성능 ===")
val_metrics = best_model.val(data=f"{DST_DIR}/data.yaml", split="val")
print(f"  mAP50:    {val_metrics.box.map50:.4f}")
print(f"  mAP50-95: {val_metrics.box.map:.4f}")

# Test 성능 (최종 리포트용)
print("\n=== Test 성능 (최종) ===")
test_metrics = best_model.val(data=f"{DST_DIR}/data.yaml", split="test")
print(f"  mAP50:    {test_metrics.box.map50:.4f}")
print(f"  mAP50-95: {test_metrics.box.map:.4f}")

print(f"\n클래스별 AP50 (Test):")
for i, ap in enumerate(test_metrics.box.ap50):
    print(f"  {YOLO_CLASSES[i]:>8s}: {ap:.4f}")

## Step 6. 추론 테스트

In [ ]:
# ── 직접 옷 사진 업로드해서 테스트 ──
from google.colab import files
import matplotlib.pyplot as plt


print("옷 사진을 업로드하세요:")
uploaded = files.upload()

for filename in uploaded:
    results = best_model.predict(
        source=filename,
        save=True,
        project="/content/runs",
        name="my_clothes",
        conf=0.25,
        exist_ok=True,
    )

    print(f"\n{filename} 탐지 결과:")
    for box in results[0].boxes:
        cls_id = int(box.cls)
        conf = float(box.conf)
        print(f"  {YOLO_CLASSES[cls_id]} ({conf:.1%})")

    annotated = results[0].plot()  # 바운딩 박스 그려진 이미지 (numpy 배열)
    plt.figure(figsize=(10, 8))
    plt.imshow(annotated[:, :, ::-1])  # BGR → RGB 변환
    plt.axis('off')
    plt.show()

## Step 7. 모델 저장 및 다운로드

In [ ]:
# ── Google Drive에 모델 저장 ──
save_dir = "/content/drive/MyDrive/fashioncody/models"
os.makedirs(save_dir, exist_ok=True)

shutil.copy(f"{result_dir}/weights/best.pt", f"{save_dir}/yolov8_clothing_best.pt")

for fname in ["results.png", "confusion_matrix.png"]:
    src = f"{result_dir}/{fname}"
    if os.path.exists(src):
        shutil.copy(src, f"{save_dir}/{fname}")

print(f"모델 저장 완료: {save_dir}/yolov8_clothing_best.pt")
print("Google Drive에 저장 → 런타임 끊겨도 유지됩니다.")

In [ ]:
# ── PC로 직접 다운로드 ──
files.download(f"{result_dir}/weights/best.pt")
print("\n다운로드한 best.pt를 프로젝트 루트 폴더에 넣으세요:")
print("28th-project-fashioncody/best.pt")

---
## 완료!

다운로드한 `best.pt`를 프로젝트 루트에 넣고 로컬에서 사용:

```python
from ultralytics import YOLO

model = YOLO("best.pt")
results = model.predict("옷사진.jpg")

for box in results[0].boxes:
    cls = int(box.cls)    # 0:top, 1:bottom, 2:outer, 3:dress, 4:acc
    conf = float(box.conf)
    print(f"{cls} {conf:.1%}")
```

전체 파이프라인 실행:
```bash
python pipeline.py --image "옷사진.jpg" --model "best.pt"
```